In [7]:
import sqlite3
import pandas as pd

## Step 1: Create a new SQLITE3 Database and Load Data from .sql File

In [8]:
# Step 1: Create a new SQLite database or connect to an existing one
SQLITE_DB_NAME = "data/country_club.db"
conn = sqlite3.connect(SQLITE_DB_NAME)
conn.close()

In [9]:
# Step 2: Load the .sql file into SQLite

# Connect to the database
conn = sqlite3.connect(SQLITE_DB_NAME)
cursor = conn.cursor()

# Load the .sql file and execute its commands
SQL_FILE_PATH = "data/country_club.sql"

with open(SQL_FILE_PATH, "r") as file:
    sql_script = file.read()

cursor.executescript(sql_script)

# Commit and close
conn.commit()
conn.close()

## Step 2: Connect to the Database

In [10]:
# Connect to the database
conn = sqlite3.connect(SQLITE_DB_NAME)
cursor = conn.cursor()

# Check the database file path (useful for verification)
cursor.execute("PRAGMA database_list;")
print('DB list:', cursor.fetchall())

# Get list of tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("\nTables in the database:", tables)

# Show table schema
cursor.execute("PRAGMA table_info(Facilities);")
table_info = cursor.fetchall()

print('\nPrinting table_info: Facilities:')
for col in table_info:
    print(col)
print('\ncolumn names:', [col[1] for col in table_info])

# Query data
cursor.execute("SELECT * FROM Facilities LIMIT 5")
rows = cursor.fetchall()

# Print results
print('\nPrinting values of the table Facilities:')
for row in rows:
    print(row)

# Close the connection
conn.close()

DB list: [(0, 'main', '/Users/shwetamehta/springboard/bootcamp/unit_8_sql_case_study_country_club/data/country_club.db')]

Tables in the database: [('Bookings',), ('Facilities',), ('Members',)]

Printing table_info: Facilities:
(0, 'facid', 'tinyint(4)', 0, 'NULL', 0)
(1, 'name', 'varchar(15)', 0, 'NULL', 0)
(2, 'membercost', 'decimal(2,1)', 0, 'NULL', 0)
(3, 'guestcost', 'decimal(3,1)', 0, 'NULL', 0)
(4, 'initialoutlay', 'mediumint(9)', 0, 'NULL', 0)
(5, 'monthlymaintenance', 'smallint(6)', 0, 'NULL', 0)
(6, 'expense_label', 'varchar(15)', 0, 'NULL', 0)

column names: ['facid', 'name', 'membercost', 'guestcost', 'initialoutlay', 'monthlymaintenance', 'expense_label']

Printing values of the table Facilities:
(0, 'Tennis Court 1', 5, 25, 10000, 200, 'expensive')
(1, 'Tennis Court 2', 5, 25, 8000, 200, 'expensive')
(2, 'Badminton Court', 0, 15.5, 4000, 50, 'cheap')
(3, 'Table Tennis', 0, 5, 320, 10, 'cheap')
(4, 'Massage Room 1', 9.9, 80, 4000, 3000, 'expensive')


# Homework: Part 2 SQLite

In [11]:
# Create a function to run user specified queries on the database
def _execute_query(query, DB_NAME=SQLITE_DB_NAME):
    
    # Connect to the database
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    
    # Execute the query and load the results into a Pandas DataFrame
    df = pd.read_sql_query(query, conn)
    
    # Print the DataFrame
    # print(df)
    
    conn.close()

    return df

<span style="color: blue;">
Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!
<span>

In [12]:
query_10_solution_i = """
  SELECT
      f.name AS facility_name,
      SUM((CASE 
          WHEN m.firstname = 'GUEST' THEN f.guestcost
          ELSE f.membercost
          END) * b.slots) AS facility_revenue

  FROM Bookings b
      INNER JOIN Facilities f
          ON b.facid = f.facid
      INNER JOIN Members m
          ON m.memid = b.memid

  GROUP BY 
      f.name

  HAVING 
      facility_revenue < 1000

  ORDER BY
      2 DESC,
      1 ASC
  ;
  """

In [13]:
_execute_query(query_10_solution_i)

,facility_name,facility_revenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


In [14]:
query_10_solution_ii = """
  SELECT
      f.name AS facility_name,
      SUM((CASE 
          WHEN m.firstname = 'GUEST' THEN f.guestcost
          ELSE f.membercost
          END) * b.slots) AS facility_revenue

  FROM Bookings b
      INNER JOIN Facilities f
          ON b.facid = f.facid
      INNER JOIN Members m
          ON m.memid = b.memid

  GROUP BY 
      f.name

  ORDER BY
      2 DESC,
      1 ASC
  ;
  """

In [15]:
df_10 = _execute_query(query_10_solution_ii)
df_10[df_10.facility_revenue < 1000].sort_values(by='facility_revenue', ascending=False)

,facility_name,facility_revenue
6,Pool Table,270.0
7,Snooker Table,240.0
8,Table Tennis,180.0


<span style="color: blue;">
Q11: Produce a report of members and who recommended them in alphabetic surname, firstname order
<span>

In [16]:
query_11 = """
  SELECT
    m1.surname || ' ' || m1.firstname AS member_name,
    m2.surname || ' ' || m2.firstname AS recommender_name
  
  FROM Members m1
    LEFT JOIN Members m2 ON m1.recommendedby = m2.memid
    
  WHERE
    m1.firstname <> 'GUEST'
  
  ORDER BY 1, 2
  ; """

In [17]:
_execute_query(query_11)

,member_name,recommender_name
0,Bader Florence,Stibbons Ponder
1,Baker Anne,Stibbons Ponder
2,Baker Timothy,Farrell Jemima
3,Boothe Tim,Rownam Tim
4,Butters Gerald,Smith Darren
5,Coplin Joan,Baker Timothy
6,Crumpet Erica,Smith Tracy
7,Dare Nancy,Joplette Janice
8,Farrell David,None
9,Farrell Jemima,None


<span style="color: blue;"> 
Q12: Find the facilities usage by member, but not guests
</span>

In [18]:
query_12 = """
  SELECT
      f.name AS facility_name,
      SUM(b.slots) AS facility_usage_in_slots,
      SUM(b.slots) * 0.5 AS facility_usage_in_hours

  FROM Bookings b
      INNER JOIN Facilities f
          ON b.facid = f.facid

  WHERE
    b.memid <> (SELECT memid FROM Members WHERE firstname = 'GUEST')

  GROUP BY 
      f.name

  ORDER BY
      2 DESC,
      1 ASC
  ;
  """

In [19]:
_execute_query(query_12)

,facility_name,facility_usage_in_slots,facility_usage_in_hours
0,Badminton Court,1086,543.0
1,Tennis Court 1,957,478.5
2,Massage Room 1,884,442.0
3,Tennis Court 2,882,441.0
4,Snooker Table,860,430.0
5,Pool Table,856,428.0
6,Table Tennis,794,397.0
7,Squash Court,418,209.0
8,Massage Room 2,54,27.0


<span style="color: blue;"> 
Q13: Find the facilities usage by month, but not guests
</span>

In [20]:
query_13 = """
  SELECT
    strftime('%Y-%m', b.starttime) AS year_month,
    SUM(b.slots) AS facility_usage_in_slots,
    SUM(b.slots) * 0.5 AS facility_usage_in_hours

  FROM Bookings b

  WHERE
    b.memid <> (SELECT memid FROM Members WHERE firstname = 'GUEST')

  GROUP BY 
      1

  ORDER BY
      1 ASC
  ;
"""

In [21]:
_execute_query(query_13)

,year_month,facility_usage_in_slots,facility_usage_in_hours
0,2012-07,1061,530.5
1,2012-08,2531,1265.5
2,2012-09,3199,1599.5
